# Working with APIs in Python

You may have heard of an "API", and you may even have a notion that APIs might be a tool for getting access to interesting information, but they're difficult to use without any coding. With a Jupyter notebook, you still need to write code, but you don't need to write one perfect script to do exactly what you want the first time around. You can experiment and look at what each line of code does.

As an example, the Harvard Art Museums provide access to a ton of information about their collections, available through a pretty simple API.

Here's some [API documentation](https://github.com/harvardartmuseums/api-docs/blob/master/sections/object.md) and a [sample API query](https://api.harvardartmuseums.org/object?q=totalpageviews:0&size=10&apikey=6dd7b080-3635-11ea-851d-67c3f7f3cf76).



In [2]:
import requests
import random
import csv

## Authentication

You'll have to authenticate each request to the Harvard Art Museum API with an API key. Other APIs may require different kinds of authentication (sometimes very complicated auth! Look for libraries at that point), but HAM has some pretty simple authentication, which makes things easy for us.

You can sign up for a key [here](https://www.harvardartmuseums.org/collections/api). Documentation for the entire API is hosted on GitHub and can be viewed [here](https://github.com/harvardartmuseums/api-docs).

In [3]:
APIKEY = "6dd7b080-3635-11ea-851d-67c3f7f3cf76"

## Basic request

We're going to start off with a basic request to the API. This API, like many others, has a variety of endpoints, each with their own url, slightly modified from a base url. We'll worry about the general case in a bit, for now let's look at a basic API request.

In this example, we'll re-create the first example in the [Object endpoint documentation](https://github.com/harvardartmuseums/api-docs/blob/master/sections/object.md), which will give each of you the records for 10 objects that have never been viewed online in the museum's collections.

In [78]:
url = "https://api.harvardartmuseums.org/object"
parameters = {
    "size":100,
    "apikey":APIKEY,
    "sort":'totalpageviews',
    "sortorder":"desc"
}
R = requests.get(url,params=parameters)
results = R.json()
results

{'info': {'next': 'https://api.harvardartmuseums.org/object?size=100&apikey=6dd7b080-3635-11ea-851d-67c3f7f3cf76&sort=totalpageviews&sortorder=desc&page=2',
  'page': 1,
  'pages': 2340,
  'totalrecords': 233927,
  'totalrecordsperquery': 100},
 'records': [{'accessionmethod': 'Bequest',
   'accessionyear': 1951,
   'accesslevel': 1,
   'century': '19th century',
   'classification': 'Paintings',
   'classificationid': 26,
   'colorcount': 10,
   'colors': [{'color': '#64af7d',
     'css3': '#5f9ea0',
     'hue': 'Green',
     'percent': 0.2979781420765,
     'spectrum': '#4fb94f'},
    {'color': '#64c896',
     'css3': '#66cdaa',
     'hue': 'Green',
     'percent': 0.21289617486339,
     'spectrum': '#47b853'},
    {'color': '#323219',
     'css3': '#2f4f4f',
     'hue': 'Brown',
     'percent': 0.19814207650273,
     'spectrum': '#3db657'},
    {'color': '#7d7d4b',
     'css3': '#696969',
     'hue': 'Green',
     'percent': 0.056775956284153,
     'spectrum': '#6cbd45'},
    {'colo

In [79]:
from IPython.core.display import display, HTML

records = results['records']
htmlOutput = "<table><tr><th>Views</th><th>Title</th></tr>"
for record in records:
    htmlOutput += f"<tr><td>{record['totalpageviews']}</td><td><a href=\"{record['url']}\" target='blank'>{record['title']}</a></td></tr>"
htmlOutput += "</table>"
display(HTML(htmlOutput))

Views,Title
31749,Self-Portrait Dedicated to Paul Gauguin
23634,The Gare Saint-Lazare: Arrival of a Train
16994,"Bahram Gur Fights the Horned Wolf (painting, verso; text, recto), illustrated folio from a manuscript of the Great Ilkhanid Shahnama (Book of Kings)"
15370,"Odalisque, Slave, and Eunuch"
13891,"Jeanne-Antoinette Poisson, Marquise de Pompadour"
13301,"A Mother and Child and Four Studies of Her Right Hand, 1904; verso: Self-Portrait Standing, 1903"
12400,"Red Boats, Argenteuil"
10890,"Court of Gayumars (painting, recto; text, verso), folio from a manuscript of the Shahnama by Firdawsi"
9456,Light Prop for an Electric Stage (Light-Space Modulator)
9035,Self-Portrait in Tuxedo


In [80]:
colors = []
for entry in records:
    if "colors" in entry:
        for c in entry['colors']:
            if c['color'] not in colors:
                colors.append(c['color'])
            else:
                print(f"{c['color']} is a repeat")

#4b647d is a repeat
#967d64 is a repeat
#af967d is a repeat
#4b4b32 is a repeat
#4b647d is a repeat
#191919 is a repeat
#96afaf is a repeat
#4b647d is a repeat
#64644b is a repeat
#af7d4b is a repeat
#c8c8c8 is a repeat
#afafaf is a repeat
#4b4b4b is a repeat
#969696 is a repeat
#191919 is a repeat
#191919 is a repeat
#afafaf is a repeat
#323232 is a repeat
#969696 is a repeat
#c8c8c8 is a repeat
#7d7d7d is a repeat
#4b4b4b is a repeat
#646464 is a repeat
#324b64 is a repeat
#4b4b4b is a repeat
#af967d is a repeat
#c8af7d is a repeat
#7d7d64 is a repeat
#7d644b is a repeat
#646464 is a repeat
#323232 is a repeat
#7d3219 is a repeat
#4b4b32 is a repeat
#644b4b is a repeat
#7d7d64 is a repeat
#191919 is a repeat
#96967d is a repeat
#e1e1e1 is a repeat
#7dafaf is a repeat
#321919 is a repeat
#323232 is a repeat
#64644b is a repeat
#7d7d64 is a repeat
#af967d is a repeat
#c8c8af is a repeat
#64644b is a repeat
#7d7d64 is a repeat
#4b4b4b is a repeat
#969696 is a repeat
#323232 is a repeat


In [131]:
len(colors)

400

In [133]:
random.shuffle(colors)
htmlOutput = '<table style="-webkit-text-stroke: 1.5px #000000; color:#ffffff; font-size:26px; font-family:Impact;"><tr>'
counter = 0
num_columns = 8
for c in colors:
    htmlOutput += f'<td style="background-color:{c}; padding:25px;">{c}</td>'
    if counter >= num_columns - 1:
        htmlOutput += "</tr><tr>"
        counter = 0
    else:
        counter += 1
htmlOutput += "</tr></table>"
display(HTML(htmlOutput))

#fae1af,#641964,#4b7de1,#c84b32,#19197d,#faaf32,#fae196,#960000
#af0019,#e1e17d,#64af7d,#af9696,#006464,#4b6464,#e19619,#e1e1af
#969696,#7d1919,#4b0000,#4b7dc8,#af9664,#197d7d,#e16419,#003232
#964b32,#7d0000,#c86432,#c84b4b,#c83232,#fae17d,#4b7daf,#af7d32
#e1e1e1,#1964af,#fafae1,#64c896,#e1af4b,#af7d7d,#fac87d,#af9619
#644b19,#c8c800,#c8af96,#c89632,#6496c8,#fa004b,#190019,#af6464
#191919,#af96c8,#967d32,#fafa64,#196496,#649664,#afaf4b,#e1af64
#7d4b4b,#af4b19,#e1967d,#64afaf,#191932,#96967d,#964b7d,#7d9696
#7d1900,#7d4b64,#967d19,#4b6496,#644b00,#c896af,#4b4b64,#4b7d96
#af3219,#c8afaf,#001964,#fafafa,#e17d7d,#323219,#e1e1fa,#e1e14b
#327d32,#fae119,#fafac8,#7d3200,#7d6432,#c8967d,#964b4b,#afaf64


In [134]:
def color_search(color):
    color = str(color)
    if color[0] != "#":
        color = "#" + color
    Q = {
        "apikey": APIKEY,
        "color": color,
        "size": 10,
        "sort": f"random:{random.randint(0,9999999)}"
    }
    R = requests.get("https://api.harvardartmuseums.org/object", params=Q)
    info  = R.json()['info']
    records = R.json()['records']
    
    for entry in records:
        if "colors" in entry:
            for c in entry['colors']:
                if c['color'] not in colors:
                    colors.append(c['color'])
                    print("new color found")
    
    output = f"<p>Search returned {info['totalrecords']} results. Showing first {info['totalrecordsperquery']}.</p><table>"
    for r in records:
        row = f'<tr><td><a href="{r["url"]}" target="_blank">{r["title"]}</td><td style="width:300px;">'
        for color in r['colors']:
            row += f'<div style="background-color:{color["color"]}; width:{color["percent"] * 100}%; float:left;">&nbsp;</div>'
        row += "</td>"
        try:
            imgUrl = r["images"][0]["iiifbaseuri"].replace("iiif","view")
            row += f'<td><a href="{imgUrl}" target="_blank"><img style="height:100px;" src="{imgUrl}"></img></a></td>'
        except IndexError:
            row += "<td></td>"
        row += "</tr>"
        output += row
    output += "</table>"
    display(HTML(output))
    return 0

In [146]:
color_search("fae1e1")

new color found
new color found


Portrait of a Man,,
Peasant Couple Feasting,,
Bronze Statue of a Genie,,
"Illustrated Book of Professions with Kyōka (Kyōka Yamato jinbutsu), 1st of 7 Volumes",,
St. Peter,,
Mars and Venus Surprised by Vulcan,,
Sketches of the Ten Kings of Hell (Jūō zu),,
Omar the Tent Maker,,
Initial H,,
"Gudarz Falls under the Spell of Susan (painting recto; text verso of folio 267), Illustrated folio from a manuscript of the Shahnama by Firdawsi",,


0

In [147]:
import csv

In [151]:
with open("colors.csv", 'w') as fp:
    writer = csv.writer(fp)
    for c in colors:
        writer.writerow([c])

In [152]:
newColors = []
with open("colors.csv", "r") as fp:
    reader = csv.reader(fp)
    for row in reader:
        newColors.append(row[0])

In [154]:
len(newColors)

413